In [97]:
pip install deap update_checker tqdm stopit xgboost

Note: you may need to restart the kernel to use updated packages.


In [98]:
! pip install tpot

In [99]:
from tpot import TPOTClassifier
import joblib

In [81]:
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
import pandas as pd
import numpy as np 
import os
import soundfile as sf
import os
from scipy.stats import randint, uniform, loguniform
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, auc
from sklearn.ensemble import RandomForestClassifier

### 1- Definitions

In [82]:
def get_train_test(complete_df):
    #split, process and return  X_train, X_test, y_train, y_test
    df_train, df_test = train_test_split(complete_df, test_size=0.3,stratify=complete_df['LABEL'])
    
    count = df_train['LABEL'].value_counts()
    count[count.isin([min(count)])].index[0]
    under_sampled = df_train[df_train['LABEL']==count[count.isin([max(count)])].index[0]].sample(min(count))
    balanced_df = pd.concat([under_sampled, df_train[df_train['LABEL']==count[count.isin([min(count)])].index[0]]], axis=0)
    X_train = balanced_df[balanced_df.columns[0:-1]]
    X_test = df_test[balanced_df.columns[0:-1]]
    y_label = balanced_df['LABEL']
    y_label_test = df_test['LABEL']
    le = LabelEncoder()
    le.fit(y_label)
    # le.classes_
    y_train = le.transform(y_label)
    y_test = le.transform(y_label_test)
    return X_train, X_test, y_train, y_test


In [83]:
def test_model(svm_XGB_best2):
    #test model with our tests
    
    #read tests
    new_test_taylor_r = pd.read_csv('Data/extracted_segment_10_extracted_segment_Taylor Swift Talks Record-Breaking Midnights Album, Music Video Cameos and Easter Eggs.csv')
    new_test_bidenf = pd.read_csv('Data/audio_features_biden_AI.csv')
    new_test_bidenr = pd.read_csv('Data/audio_features_biden.csv')
    new_test_el = pd.read_csv('Data/Elise.csv')
    new_test_yr1 = pd.read_csv('Data/Youssef.csv')
    new_test_yr2 = pd.read_csv('Data/Youssef_2.csv')
    new_test_yf1 = pd.read_csv('Data/Youssef_Eric Cartman.csv')
    new_test_yf2 = pd.read_csv('Data/Youssef_2_Female.csv')
    new_test_yf3  = pd.read_csv('Data/Youssef_2_Male Reggaeton.csv')
    new_test_MF_f  = pd.read_csv('Data/Morgan Freeman_fake.csv')
    new_test_MF_r  = pd.read_csv('Data/Morgan Freeman_real.csv')
    new_test_And_r = pd.read_csv('Data/Andrea.csv')
    new_test_And_f = pd.read_csv('Data/Andrea_to_Youss.csv')
    
    #make y_true
    y_real = np.ones((1000,))
    y_fake = np.zeros((1000,))
    
    #predict and get accuray score
    pred = svm_XGB_best2.predict(new_test_taylor_r)
    taylor_r = accuracy_score(y_real[:len(pred)],pred)
    print('Real Taylor : ' , taylor_r)
    
    pred = svm_XGB_best2.predict(new_test_bidenf[new_test_bidenf.columns[0:-2]])
    bidenf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Biden : ' , bidenf)

    pred = svm_XGB_best2.predict(new_test_bidenr[new_test_bidenf.columns[0:-2]])
    bidenr=accuracy_score(y_real[:len(pred)],pred)
    print('Real Biden : ' , bidenr)

    pred = svm_XGB_best2.predict(new_test_el)
    eliser=accuracy_score(y_real[:len(pred)],pred)
    print('Real Elise : ' , eliser)

    pred = svm_XGB_best2.predict(new_test_yr1)
    youssr1 = accuracy_score(y_real[:len(pred)],pred)
    print('Real Youss_1 : ' , youssr1)

    pred = svm_XGB_best2.predict(new_test_yr2)
    youssr2 = accuracy_score(y_real[:len(pred)],pred)
    print('Real Youss_2 : ' , youssr2)

    pred = svm_XGB_best2.predict(new_test_yf1)
    youssf1 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_1 : ' , youssf1)

    pred = svm_XGB_best2.predict(new_test_yf2)
    youssf2 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_2 : ' , youssf2)

    pred = svm_XGB_best2.predict(new_test_yf3)
    youssf3 = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Youss_3 : ' , youssf3)

    pred = svm_XGB_best2.predict(new_test_MF_f)
    morganf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Morgan : ' , morganf)

    pred = svm_XGB_best2.predict(new_test_MF_r)
    morganr = accuracy_score(y_real[:len(pred)],pred)
    print('Real Morgan : ' , morganr)

    pred = svm_XGB_best2.predict(new_test_And_f)
    andf = accuracy_score(y_fake[:len(pred)],pred)
    print('Fake Andrea : ' , andf)

    pred = svm_XGB_best2.predict(new_test_And_r)
    andr = accuracy_score(y_real[:len(pred)],pred)
    print('Real Andrea : ' , andr)

    

### 3- Estimators

#### XGBOOST

### Adding asvc LA 2019 features

In [3]:
asvc_features = pd.read_csv('Data/asvc_features.csv')
full_df = pd.read_csv('Data/full_df.csv')
la_2029_bona = pd.read_csv('Data/la_2029_bona.csv')
la_2029_spoof= pd.read_csv('Data/la_2029_spoof.csv')
asvc_features


,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.581642,0.003070,2199.394712,1951.288198,4489.428156,0.101729,-465.42130,125.339874,-69.007560,70.611090,...,-8.866219,7.612215,-12.527738,9.243306,-17.393263,6.024668,-14.883923,-1.570087,-11.686265,LA_D_8848915
1,0.325574,0.093229,2345.454133,1599.777611,3984.376665,0.152854,-236.49667,100.673485,-32.507786,49.088180,...,-10.290752,-2.104480,-12.366513,-0.074342,-20.750038,-0.219439,-13.774378,-12.227905,-15.365830,LA_D_8848915
2,0.315612,0.107650,1792.799111,1426.987016,3165.870250,0.124190,-233.42374,128.119860,-33.989710,45.122196,...,-4.690394,-8.381293,-15.633381,-2.263502,-20.402868,-5.021841,-14.528839,-10.487343,-13.769796,LA_D_8848915
3,0.539373,0.001211,1397.669671,1798.662060,3206.245006,0.058805,-601.98300,157.632460,-29.607780,64.629550,...,-18.353567,-2.086941,-21.357689,1.418289,-19.528841,2.797719,-17.868378,-0.484152,-12.906013,LA_D_4477471
4,0.309486,0.177945,1369.867761,1431.153215,2583.005593,0.078857,-275.83110,104.976555,-20.991016,46.207660,...,-12.673950,-6.094771,-14.902205,0.231349,-26.601341,-16.148132,-9.064847,-9.427711,-17.687447,LA_D_4477471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338744,0.371957,0.108856,1874.572218,1644.345669,3483.484996,0.100042,-267.69348,112.351730,-38.577652,28.142616,...,-9.952971,-4.968857,-4.419859,-3.030780,-23.952742,2.336856,-18.854225,-8.214225,-4.492851,c/LA_T_51403
338745,0.334515,0.126682,1887.510015,1628.934264,3496.209162,0.117687,-200.80446,128.001220,-38.918960,32.557312,...,-18.347965,1.964419,-15.538897,-1.297084,-12.130425,-5.399279,-15.976752,-13.512214,-8.315792,c/LA_T_83730
338746,0.268613,0.169597,1588.532926,1506.019870,3034.958163,0.081332,-194.92460,117.109030,-29.384462,49.451450,...,-17.681974,0.498010,-7.854861,-2.573860,-16.794678,-10.481019,-14.802436,-13.050943,-10.861348,c/LA_T_83730
338747,0.410026,0.138760,1677.571697,1590.618139,3213.096480,0.094649,-205.35316,128.130100,-34.798000,44.976448,...,-14.185932,-1.863424,-9.754604,-3.552697,-16.958033,-6.803551,-13.341378,-4.011154,-12.574131,c/LA_T_96703


In [34]:
full_df.LABEL.value_counts()

LABEL
FAKE    26194
REAL    15890
Name: count, dtype: int64

In [18]:
la_2029_spoof

,name_1,name_2,label
0,LA_0069,LA_D_1008730,spoof
1,LA_0069,LA_D_1034049,spoof
2,LA_0069,LA_D_1048723,spoof
3,LA_0069,LA_D_1067573,spoof
4,LA_0069,LA_D_1091909,spoof
...,...,...,...
108973,LA_0009,LA_E_3635712,spoof
108974,LA_0010,LA_E_4078898,spoof
108975,LA_0038,LA_E_5085671,spoof
108976,LA_0012,LA_E_4926022,spoof


In [7]:
#adding labels to asvc_features : 
asvc_features.LABEL[asvc_features.LABEL.isin(la_2029_spoof.name_2)] = 'FAKE'
asvc_features.LABEL[asvc_features.LABEL.isin(la_2029_bona.name_2)] = 'REAL'
asvc_fake = asvc_features[asvc_features.LABEL== 'FAKE']
asvc_real = asvc_features[asvc_features.LABEL== 'REAL']

transfer = pd.concat([asvc_fake,asvc_real])
transfer.LABEL.value_counts()
transfer.to_csv('Asvc_2019_features.csv',index = False)

/var/tmp/ipykernel_3935/473903352.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asvc_features.LABEL[asvc_features.LABEL.isin(la_2029_spoof.name_2)] = 'FAKE'
/var/tmp/ipykernel_3935/473903352.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asvc_features.LABEL[asvc_features.LABEL.isin(la_2029_bona.name_2)] = 'REAL'


In [42]:
full_asvc = pd.concat([asvc_fake,asvc_real,full_df])
full_asvc.to_csv('full_df_2013-12.csv',index = False)


In [6]:
full_df = pd.read_csv('Data/full_df_2023-12-08.csv')

In [7]:
full_df.LABEL.value_counts()

LABEL
1    140403
0     25528
Name: count, dtype: int64

In [18]:
X_train, X_test, y_train, y_test = get_train_test(full_df)


0    17869
1    17869
Name: count, dtype: int64

In [13]:
grid = {'n_estimators': randint(1, 400),
        'max_depth': randint(1, 13),  
        'learning_rate': loguniform (0.001, 0.5)
       }
bst = XGBClassifier()
# Instantiate Grid Search
search = RandomizedSearchCV(
    bst,
    grid, 
    scoring='precision',
    n_iter=100,  
    cv=5, n_jobs=-1
)

search.fit(X_train, y_train)
search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2319503520076279,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=266, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [15]:
grid = {'n_estimators': [250,266, 350,300,400],
        'max_depth': range(9,11),  
        'learning_rate': [0.2, 0.22, 0.2319503520076279, 0.24, 0.25, 0.26, 0.27, 0.28]
       }
bst = XGBClassifier()
# # Instantiate Grid Search
search = GridSearchCV(
     bst,
     grid, 
     scoring='precision',
     cv=5, n_jobs=-1
)
search.fit(X_train, y_train)
search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2319503520076279,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [19]:
# svm_XGB_best2 = XGBClassifier(learning_rate=0.2, max_depth=9, n_estimators=450)
svm_XGB_best2 = XGBClassifier(n_estimators=400, max_depth=9, learning_rate=0.2319503520076279, objective='binary:logistic')

In [17]:
cross_val_score(svm_XGB_best2,X_train,y_train,cv=10,scoring='accuracy').mean()


0.8899770804819018

In [18]:
cross_val_score(svm_XGB_best2,X_train,y_train,cv=5,scoring='precision',n_jobs=-1).mean()


0.8712522290812783

In [19]:
cross_val_score(svm_XGB_best2,X_train,y_train,cv=5, n_jobs=-1,scoring='recall').mean()


0.9076613273931156

In [20]:
svm_XGB_best2.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2319503520076279,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [21]:
y_pred = svm_XGB_best2.predict(X_test)

accuracy_score(y_test, y_pred)

0.9053435114503817

In [22]:
precision_score(y_test, y_pred)

0.974432466708941

In [23]:
recall_score(y_test, y_pred)

0.9120628665036442

### Taking out the phone asvc data

In [71]:
df_asvc2021 = pd.read_csv('Data/new_df_ASV_DF_Elise.csv')
full_df = pd.read_csv('Data/full_df.csv')


In [72]:
df_asvc2021.LABEL = df_asvc2021.LABEL.map({ 1 : 'REAL', 0 : 'FAKE'} )

In [73]:
new_df = pd.concat([df_asvc2021,full_df])

In [53]:
new_df.to_csv('Data/df_asvc_without_LA.csv',index=False)

In [74]:
new_df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,LABEL
0,0.306480,0.076295,1413.467433,1839.001565,2703.151079,0.053345,-232.30121,133.165650,-22.955957,8.888358,...,2.394415,-3.317231,-14.874663,-4.914529,-1.474122,-10.153106,-9.911881,-3.817368,-10.349003,REAL
1,0.477024,0.006032,2639.868324,2527.274422,5466.497248,0.117154,-383.07986,90.157790,-8.644286,15.567981,...,0.886504,-8.037273,-2.922858,-8.647303,-2.993608,-6.740992,-4.170974,-7.415453,-4.745568,REAL
2,0.423958,0.054416,1729.632773,2022.347548,3623.206121,0.079767,-287.24896,125.231820,-8.775691,22.783344,...,-2.435866,-9.924732,-2.930283,-7.618189,-2.912869,-5.278629,-3.483856,-4.521770,-0.359239,REAL
3,0.423640,0.038472,2266.627450,2006.521778,4112.597101,0.118597,-265.62040,103.924530,-9.073397,43.439182,...,-3.888492,-8.790924,2.843195,-3.305686,-1.018029,-0.356833,0.381062,-4.544253,0.707247,REAL
4,0.502441,0.036537,2135.115673,2027.611411,4083.723034,0.128285,-310.34613,113.192940,-28.261267,12.852004,...,-4.229133,-6.774048,5.878327,-2.639606,-0.157469,-2.179182,-3.610251,-6.534174,-3.241210,REAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42079,0.314406,0.046116,2698.440901,2535.992561,5354.182018,0.149414,-255.56259,87.417946,4.433178,13.824187,...,-0.741367,-4.572025,14.577326,-4.435467,-6.427833,4.626421,-0.856710,-12.452103,2.811413,REAL
42080,0.370378,0.056801,2955.994101,2328.057783,5294.231623,0.190552,-232.96104,76.125670,-1.108529,18.675050,...,2.659945,-9.495482,8.700753,-1.669142,-10.515453,0.587301,4.574445,-8.807771,4.212809,REAL
42081,0.309927,0.076453,1884.453339,2071.263641,3594.332053,0.091908,-237.58550,107.939300,-22.380022,18.590910,...,6.337625,-7.541246,11.250736,-8.606643,-7.723567,1.479188,-9.126733,-8.543749,0.512092,REAL
42082,0.385543,0.030032,2407.104292,2433.123626,4915.932395,0.120594,-269.90726,91.986984,9.712902,21.076706,...,2.552168,-6.178091,6.619003,-7.936121,-7.672117,0.772831,0.610292,-10.339297,0.709354,REAL


In [75]:
X_train, X_test, y_train, y_test = get_train_test(new_df)

In [47]:
svm_XGB_best2.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2319503520076279,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [49]:
grid = {'n_estimators': randint(1, 400),
        'max_depth': randint(1, 13),  
        'learning_rate': loguniform (0.001, 0.5)
        # 'max_leaves' : 
       }
bst = XGBClassifier()
# Instantiate Grid Search
search = RandomizedSearchCV(
    bst,
    grid, 
    scoring='precision',
    n_iter=100,  
    cv=5, n_jobs=-1
)

search.fit(X_train, y_train)
search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.015363601183606038,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=209, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [50]:
grid = {'n_estimators': [200,209,300,350,400],
        'max_depth': range(5,8),  
        'learning_rate': [0.015363601183606038, 0.02, 0.2]
       }
bst = XGBClassifier()
# # Instantiate Grid Search
search = GridSearchCV(
     bst,
     grid, 
     scoring='precision',
     cv=5, n_jobs=-1
)
search.fit(X_train, y_train)
search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.015363601183606038,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [54]:
svm_XGB_best2 = search.best_estimator_

In [56]:
svm_XGB_best2.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.015363601183606038,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [60]:
y_pred = svm_XGB_best2.predict(X_test)

accuracy_score(y_test, y_pred)

0.6680163954788225

In [61]:
test_model(svm_XGB_best2)

Real Taylor :  0.0
Fake Biden :  0.975
Real Biden :  0.9722222222222222
Real Elise :  0.043478260869565216
Real Youss_1 :  0.0
Real Youss_2 :  0.03333333333333333
Fake Youss_1 :  1.0
Fake Youss_2 :  1.0
Fake Youss_3 :  0.9666666666666667
Fake Morgan :  0.9523809523809523
Real Morgan :  0.022988505747126436


In [59]:
full_df.columns

Index(['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth',
       'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4',
       'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11',
       'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18',
       'mfcc19', 'mfcc20', 'LABEL'],
      dtype='object')

In [95]:
import joblib
filename = 'XGB_test'
joblib.dump(svm_XGB_best2,f'models/{filename}_no_ASVC')


['models/XGB_test_no_ASVC']

### Teapot without ASVC data

In [96]:
complete_df= pd.read_csv('Data/New_features.csv')
elise_df = pd.read_csv('Data/additional_data_elise.csv')
youss_df = pd.read_csv('Data/Extra_features.csv')
elise_df = elise_df[elise_df.columns[1:]]
elise_df['LABEL'] = 'REAL'
full_df = pd.concat([complete_df,elise_df,youss_df])

In [100]:
pipeline_optimizer = TPOTClassifier(n_jobs=-1,warm_start=True,verbosity=2)


In [101]:
X_train, X_test, y_train, y_test = get_train_test(full_df)

In [102]:
pipeline_optimizer.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9526206671869198

Generation 2 - Current best internal CV score: 0.9526206671869198

Generation 3 - Current best internal CV score: 0.9526206671869198

Generation 4 - Current best internal CV score: 0.9550931228075493

Generation 5 - Current best internal CV score: 0.9550931228075493

Generation 6 - Current best internal CV score: 0.9550931228075493

Generation 7 - Current best internal CV score: 0.9550931228075493

Generation 8 - Current best internal CV score: 0.9550931228075493

Generation 9 - Current best internal CV score: 0.9550931228075493

Generation 10 - Current best internal CV score: 0.9571608011900162

Generation 11 - Current best internal CV score: 0.9571608011900162

Generation 12 - Current best internal CV score: 0.9571608011900162

Generation 13 - Current best internal CV score: 0.9571608011900162

Generation 14 - Current best internal CV score: 0.9571608011900162

Generation 15 - Current best internal CV score: 0.957160

TPOTClassifier(n_jobs=-1, verbosity=2, warm_start=True)

In [103]:
print(pipeline_optimizer.score(X_test, y_test))


0.9729130365911611


In [104]:
pipeline_optimizer.export('tpot_exported_pipeline_no_asvc.py')


In [105]:
import pickle
with open('./test_tpot_notebook_withoutLA.pkl','wb') as f:
    pickle.dump(pipeline_optimizer.fitted_pipeline_, f)